### Imports

In [1]:
#Music21 is a python library making analysis of music accessible and fun. It supports integration with popular formats such as MIDI, MusicXML, Lilypond, and others. It's also well integrated with The Elvis Project, enabling users to import large volumes of music for easy analysis. Music21 is a great platform for musicologists and machine learning researchers alike to explore patterns and structure in music.
#About this Project :
#In this project, we will be creating an Automatic Music Generation
# model using LSTM. We will fetch notes from all music files
#  which will then be fed into the model for prediction.
#  Then finally we will create a MIDI file using these predicted notes.
from music21 import converter, instrument, note, chord, stream
import glob
import pickle
import numpy as np
from keras.utils import np_utils

## Read a Midi File

In [2]:
midi = converter.parse("midi_songs/EyesOnMePiano.mid") #package converter from music 21

In [3]:
midi

<music21.stream.Score 0x11f36400eb0>

In [4]:
midi.show('midi')

In [5]:
midi.show('text') #whole song in text format  container -> subcontainer -> notes notes chords
#[[[N,C][N,C]]]

{0.0} <music21.metadata.Metadata object at 0x11f36400e80>
{0.0} <music21.stream.Part 0x11f36401a20>
    {0.0} <music21.stream.Measure 1 offset=0.0>
        {0.0} <music21.instrument.Piano 'Staff: Piano'>
        {0.0} <music21.clef.TrebleClef>
        {0.0} <music21.tempo.MetronomeMark Quarter=95.0>
        {0.0} <music21.key.Key of D major>
        {0.0} <music21.meter.TimeSignature 4/4>
        {0.0} <music21.stream.Voice 0x11f34e088b0>
            {0.0} <music21.note.Note A>
            {0.25} <music21.note.Rest 5/12ql>
            {0.6667} <music21.note.Note G>
            {1.1667} <music21.note.Rest 1/3ql>
            {1.5} <music21.note.Note D>
            {1.8333} <music21.note.Rest 1/6ql>
            {2.0} <music21.note.Note A>
            {2.25} <music21.note.Rest 16th>
            {2.5} <music21.note.Note G>
            {2.8333} <music21.note.Rest 1/6ql>
            {3.0} <music21.note.Note D>
            {3.25} <music21.note.Rest 16th>
            {3.5} <music21.note.Note A>

In [6]:
# Flat all the elements [N,N,C,N,N,N,C,C]
elements_to_parse = midi.flat.notes 

In [7]:
len(elements_to_parse)

1553

In [8]:
for e in elements_to_parse:
    print(e, e.offset)

<music21.note.Note A> 0.0
<music21.note.Note A> 0.0
<music21.note.Note A> 0.0
<music21.note.Note A> 0.25
<music21.note.Note G> 2/3
<music21.note.Note G> 2/3
<music21.note.Note F#> 1.0
<music21.note.Note F#> 1.25
<music21.note.Note D> 1.5
<music21.note.Note D> 1.5
<music21.note.Note C#> 1.75
<music21.note.Note C#> 1.75
<music21.note.Note A> 2.0
<music21.note.Note A> 2.25
<music21.note.Note G> 2.5
<music21.note.Note G> 2.5
<music21.note.Note F#> 2.75
<music21.note.Note F#> 2.75
<music21.note.Note D> 3.0
<music21.note.Note D> 3.25
<music21.note.Note A> 3.5
<music21.note.Note A> 3.5
<music21.note.Note D> 3.75
<music21.note.Note D> 3.75
<music21.note.Note A> 4.0
<music21.note.Note D> 4.0
<music21.note.Note D> 4.0
<music21.note.Note B> 4.0
<music21.note.Note G> 4.0
<music21.note.Note A> 4.25
<music21.note.Note D> 4.25
<music21.note.Note B> 4.25
<music21.note.Note G> 4.25
<music21.note.Note E> 5.0
<music21.note.Note C#> 5.0
<music21.note.Note E> 5.25
<music21.note.Note C#> 5.25
<music21.note.

In [9]:
notes_demo = [] #empty list

for ele in elements_to_parse:
    # If the element is a Note,  then store it's pitch
    if isinstance(ele, note.Note):
        notes_demo.append(str(ele.pitch))
    
    # If the element is a Chord, split each note of chord and join them with +
    elif isinstance(ele, chord.Chord):
        notes_demo.append("+".join(str(n) for n in ele.normalOrder))

In [10]:
len(notes_demo)

1553

In [11]:
isinstance(elements_to_parse[68], chord.Chord) #isinstance whether element is note or chord

False

# Preprocessing all Files

In [12]:
notes = []

for file in glob.glob("midi_songs/*.mid"):
    midi = converter.parse(file) # Convert file into stream.Score Object
    
    print("parsing %s"%file)
    
    elements_to_parse = midi.flat.notes
    
    
    for ele in elements_to_parse:
        # If the element is a Note,  then store it's pitch
        if isinstance(ele, note.Note):
            notes.append(str(ele.pitch))

        # If the element is a Chord, split each note of chord and join them with +
        elif isinstance(ele, chord.Chord):
            notes.append("+".join(str(n) for n in ele.normalOrder))

parsing midi_songs\0fithos.mid
parsing midi_songs\8.mid
parsing midi_songs\ahead_on_our_way_piano.mid
parsing midi_songs\AT.mid
parsing midi_songs\balamb.mid
parsing midi_songs\bcm.mid
parsing midi_songs\BlueStone_LastDungeon.mid
parsing midi_songs\braska.mid
parsing midi_songs\caitsith.mid
parsing midi_songs\Cids.mid
parsing midi_songs\cosmo.mid
parsing midi_songs\costadsol.mid
parsing midi_songs\dayafter.mid
parsing midi_songs\decisive.mid
parsing midi_songs\dontbeafraid.mid
parsing midi_songs\DOS.mid
parsing midi_songs\electric_de_chocobo.mid
parsing midi_songs\Eternal_Harvest.mid
parsing midi_songs\EyesOnMePiano.mid
parsing midi_songs\ff11_awakening_piano.mid
parsing midi_songs\ff1battp.mid
parsing midi_songs\FF3_Battle_(Piano).mid
parsing midi_songs\FF3_Third_Phase_Final_(Piano).mid
parsing midi_songs\ff4-airship.mid
parsing midi_songs\Ff4-BattleLust.mid
parsing midi_songs\ff4-fight1.mid
parsing midi_songs\ff4-town.mid
parsing midi_songs\FF4.mid
parsing midi_songs\ff4pclov.mid
par

C:\Users\hp\AppData\Roaming\Python\Python310\site-packages\music21\midi\translate.py:863: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=5, channel=None, data=b'Martin Rosok Copyright \xa9 1998'>; getting generic Instrument
  warnings.warn(


parsing midi_songs\ff7themep.mid
parsing midi_songs\ff8-lfp.mid
parsing midi_songs\FF8_Shuffle_or_boogie_pc.mid
parsing midi_songs\FFIII_Edgar_And_Sabin_Piano.mid
parsing midi_songs\FFIXQuMarshP.mid
parsing midi_songs\FFIX_Piano.mid
parsing midi_songs\FFVII_BATTLE.mid
parsing midi_songs\FFX_-_Ending_Theme_(Piano_Version)_-_by_Angel_FF.mid
parsing midi_songs\Fiend_Battle_(Piano).mid
parsing midi_songs\Fierce_Battle_(Piano).mid
parsing midi_songs\figaro.mid
parsing midi_songs\Finalfantasy5gilgameshp.mid
parsing midi_songs\Finalfantasy6fanfarecomplete.mid
parsing midi_songs\Final_Fantasy_7_-_Judgement_Day_Piano.mid
parsing midi_songs\Final_Fantasy_Matouyas_Cave_Piano.mid
parsing midi_songs\fortresscondor.mid
parsing midi_songs\Fyw_piano.mid
parsing midi_songs\gerudo.mid
parsing midi_songs\goldsaucer.mid
parsing midi_songs\Gold_Silver_Rival_Battle.mid
parsing midi_songs\great_war.mid
parsing midi_songs\HighwindTakestotheSkies.mid
parsing midi_songs\In_Zanarkand.mid
parsing midi_songs\JENOV

C:\Users\hp\AppData\Roaming\Python\Python310\site-packages\music21\midi\translate.py:863: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=6, channel=None, data=b'Pok\xe9mon: Elite Four (Piano)'>; getting generic Instrument
  warnings.warn(


parsing midi_songs\Rachel_Piano_tempofix.mid
parsing midi_songs\redwings.mid
parsing midi_songs\relmstheme-piano.mid
parsing midi_songs\roseofmay-piano.mid
parsing midi_songs\rufus.mid
parsing midi_songs\Rydia_pc.mid
parsing midi_songs\sandy.mid
parsing midi_songs\sera_.mid
parsing midi_songs\sobf.mid
parsing midi_songs\Still_Alive-1.mid
parsing midi_songs\Suteki_Da_Ne_(Piano_Version).mid
parsing midi_songs\thenightmarebegins.mid
parsing midi_songs\thoughts.mid
parsing midi_songs\tifap.mid
parsing midi_songs\tpirtsd-piano.mid
parsing midi_songs\traitor.mid
parsing midi_songs\ultimafro.mid
parsing midi_songs\ultros.mid
parsing midi_songs\VincentPiano.mid
parsing midi_songs\ViviinAlexandria.mid
parsing midi_songs\waltz_de_choco.mid
parsing midi_songs\Zelda_Overworld.mid
parsing midi_songs\z_aeristhemepiano.mid


In [13]:
len(notes)

60764

In [14]:
with open("notes", 'wb') as filepath: #saving file
    pickle.dump(notes, filepath)

In [15]:
with open("notes", 'rb') as f:
    notes= pickle.load(f)

In [16]:
n_vocab = len(set(notes))

In [17]:
print("Total notes- ", len(notes))
print("Unique notes- ",  n_vocab)

Total notes-  60764
Unique notes-  398


In [18]:
print(notes[100:200])

['F3', 'F2', 'F2', 'F2', 'F2', 'F2', '4+9', 'E5', '4+9', 'C5', '4+9', 'A5', '4+9', '5+9', 'F5', '5+9', 'C5', '5+9', 'A5', '5+9', '4+9', 'E5', '4+9', 'C5', '4+9', 'A5', '4+9', 'F5', '5+9', 'C5', '5+9', 'E5', '5+9', 'D5', '5+9', 'E5', '4+9', 'E-5', '4+9', 'B5', '4+9', '4+9', 'A5', '5+9', '5+9', '5+9', '5+9', 'A5', '4+9', '4+9', '4+9', '4+9', '5+9', '5+9', '5+9', '5+9', 'B4', '4+9', 'A4', '4+9', 'E5', '4+9', '4+9', 'E-5', '5+9', '5+9', '5+9', '5+9', 'E-5', '4+9', '4+9', '4+9', '4+9', '5+9', '5+9', '5+9', '5+9', 'E5', '4', 'E-5', 'C6', 'E5', '5', 'E-5', 'B5', 'E5', '6', 'E-5', 'C6', 'A5', '5', 'A4', '4', 'C5', 'E5', 'F5', 'E5', '5', 'C5', 'A4']


# Prepare Sequential Data for LSTM

In [19]:
# How many elements LSTM input should consider
sequence_length = 100

In [20]:
# All unique classes
pitchnames = sorted(set(notes))

In [21]:
# Mapping between ele to int value
ele_to_int = dict( (ele, num) for num, ele in enumerate(pitchnames) )

In [22]:
network_input = []
network_output = []

In [23]:
for i in range(len(notes) - sequence_length):
    seq_in = notes[i : i+sequence_length] # contains 100 values
    seq_out = notes[i + sequence_length]
    
    network_input.append([ele_to_int[ch] for ch in seq_in])
    network_output.append(ele_to_int[seq_out])

In [24]:
# No. of examples
n_patterns = len(network_input)
print(n_patterns)

60664


In [25]:
# Desired shape for LSTM
network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
print(network_input.shape)

(60664, 100, 1)


In [26]:
normalised_network_input = network_input/float(n_vocab)

In [27]:
# Network output are the classes, encode into one hot vector
network_output = np_utils.to_categorical(network_output)

In [28]:
network_output.shape

(60664, 398)

In [29]:
print(normalised_network_input.shape)
print(network_output.shape)

(60664, 100, 1)
(60664, 398)


# Create Model

In [30]:
from keras.models import Sequential, load_model
from keras.layers import *
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [31]:
model = Sequential()
model.add( LSTM(units=512,
               input_shape = (normalised_network_input.shape[1], normalised_network_input.shape[2]),
               return_sequences = True) )
model.add( Dropout(0.3) )
model.add( LSTM(512, return_sequences=True) )
model.add( Dropout(0.3) )
model.add( LSTM(512) )
model.add( Dense(256) )
model.add( Dropout(0.3) )
model.add( Dense(n_vocab, activation="softmax") )

In [32]:
model.compile(loss="categorical_crossentropy", optimizer="adam")

In [33]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 100, 512)          1052672   
                                                                 
 dropout (Dropout)           (None, 100, 512)          0         
                                                                 
 lstm_1 (LSTM)               (None, 100, 512)          2099200   
                                                                 
 dropout_1 (Dropout)         (None, 100, 512)          0         
                                                                 
 lstm_2 (LSTM)               (None, 512)               2099200   
                                                                 
 dense (Dense)               (None, 256)               131328    
                                                                 
 dropout_2 (Dropout)         (None, 256)               0

In [34]:
checkpoint = ModelCheckpoint("model.hdf5", monitor='loss', verbose=0, save_best_only=True, mode='min')


model_his = model.fit(normalised_network_input, network_output, epochs=100, batch_size=64, callbacks=[checkpoint])

Epoch 1/100
948/948 [==============================] - 32936s 35s/step - loss: 4.7077
Epoch 2/100
497/948 [==============>...............] - ETA: 8:01:30 - loss: 4.5868

KeyboardInterrupt: 

In [35]:
model = load_model("new_weights.hdf5")

# Predictions

In [36]:
sequence_length = 100
network_input = []

for i in range(len(notes) - sequence_length):
    seq_in = notes[i : i+sequence_length] # contains 100 values
    network_input.append([ele_to_int[ch] for ch in seq_in])

In [37]:
# Any random start index
start = np.random.randint(len(network_input) - 1)

# Mapping int_to_ele
int_to_ele = dict((num, ele) for num, ele in enumerate(pitchnames))

# Initial pattern 
pattern = network_input[start]
prediction_output = []

# generate 200 elements
for note_index in range(200):
    prediction_input = np.reshape(pattern, (1, len(pattern), 1)) # convert into numpy desired shape 
    prediction_input = prediction_input/float(n_vocab) # normalise
    
    prediction =  model.predict(prediction_input, verbose=0)
    
    idx = np.argmax(prediction)
    result = int_to_ele[idx]
    prediction_output.append(result) 
    
    # Remove the first value, and append the recent value.. 
    # This way input is moving forward step-by-step with time..
    pattern.append(idx)
    pattern = pattern[1:]

In [38]:
print(prediction_output)

['9+11+0+2', '8+11+3', '9+10+3', '9+10+3', '9+10+3', '9+1+2', '9+10+3', '9+10+3', '9+10+3', '9+10+3', '9+10+3', '9+11', '9+11', '9+11', '9+11', '8+10+3', '9+11', '8+10+3', '8+10+3', '4+5+7+9', '4+5+7+9', '9+11', 'B3', '8+10+3', '8+11', 'D2', '8+10+3', 'B1', '8+10+3', '9+11+2+5', '8+10+3', '8+10+3', '8+10+11+1', '8+10+3', '8+10+2', '8+10+3', '8+11+1', '8+10+3', '8+11+1', '8+11+1', '8+10+3', '4+5+7+9', '8+10+3', '8+11+1', '8+10+3', '3+4+8+11', '9+11+2+5', '8+11+1', '8+11', '6+10+11+1', '8+10+2', '7+9', '7+8+11', '4+5+7+9', '9+0', '11', '0+3+6+9', 'D1', '11+3+6', '11+3+6', '0+3+6+9', 'C3', '8+11+3', '11+1+4+5', '1+4+8', '8+10+3', '0', '8+10+3', '8+10+3', '11+1+4+5', '8+10+3', '8+10+2', '8+10+2', '8+10+2', '9+11', '8+10+3', '9+11', '8+10+3', '8+10+3', '9+0', '8+10+3', '8+11+1', '8+10+3', '2+4+6+7', '7+8+11', '8+10+3', '8+10+3', '8+10+2', '8+10+2', '8+10+3', '8+10+3', '7+8+11', '7+8+11', '7+8+11', '7+8+11', '7+8+11', '7+8+11', '7+9+11+2', '8+10+3', '8+10+3', 'B-1', '8+10+3', '8+10+3', '6+11

# Create Midi File

In [39]:
offset = 0 # Time
output_notes = []

for pattern in prediction_output:
    
    # if the pattern is a chord
    if ('+' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('+')
        temp_notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))  # create Note object for each note in the chord
            new_note.storedInstrument = instrument.Piano()
            temp_notes.append(new_note)
            
        
        new_chord = chord.Chord(temp_notes) # creates the chord() from the list of notes
        new_chord.offset = offset
        output_notes.append(new_chord)
    
    else:
            # if the pattern is a note
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
        
    offset += 0.5

In [40]:
# create a stream object from the generated notes
midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp = "test_output.mid")

'test_output.mid'

In [41]:
midi_stream.show('midi')